In [243]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import csv
import time

In [244]:
all_data = []

In [ ]:
#blob

X_training = []
y_training = []
header = []
c = 0
with open("DataSets/blobs_att.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if(c == 0):
            header = row[:-1]
            c+=1
            continue
        X_training.append([float(x) for x in row])
        
with open("DataSets/blobs_label.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        y_training.append(int(row[0]))
print(X_training)
print(y_training)
        
        
X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data
all_data.append([X_train, X_test, y_train, y_test,header])

In [246]:
#iris
X_training = []
y_training = []
header = []
c = 0
with open("DataSets/iris.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if(c == 0):
            header = row[:-1]
            c+=1
            continue
        X_training.append([float(x) for x in row[:-1]])
        y_training.append(row[-1])
        #print(row)

for i in range(len(y_training)):
    if(y_training[i] == "Setosa"):
        y_training[i] = 1
    if(y_training[i] == "Versicolor"):
        y_training[i] = 2
    if(y_training[i] == "Virginica"):
        y_training[i] = 3
        
X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data
all_data.append([X_train, X_test, y_train, y_test,header])


In [ ]:
#wine
X_training = []
y_training = []
header = []
c = 0
with open("DataSets/winequality-white_att.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if(c == 0):
            header = row
            c+=1
            continue
        X_training.append([float(x) for x in row[:-1]])
        y_training.append(float(row[-1]))

X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data
all_data.append([X_train, X_test, y_train, y_test,header])
print(header)
print(X_train)
print(y_training)

In [ ]:
#house
X_training = []
y_training = []
header = []
c = 0
with open("DataSets/house_att.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if(c == 0):
            header = row
            c+=1
            continue
        X_training.append([float(x) for x in row])
        
with open("DataSets/house_price.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        y_training.append(float(row[0]))

        
print(len(X_training))
print(len(y_training))
X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data
all_data.append([X_train, X_test, y_train, y_test,header])
print(X_training)
print(y_training)

In [250]:
#Sort the dataset
def sort_f(X,y,column):
    
    for i in range(len(X)-1):
        temp = i
        for j in range(i,len(X)):
            
            

            if(X[j][column]<X[temp][column]):
                temp = j
        if(temp != i):
            tmp = X[i]
            X[i] = X[temp]
            X[temp] = tmp

            tmp = y[i]
            y[i] = y[temp]
            y[temp] = tmp
    return X,y

In [254]:
class Tree:
    def __init__(self,X,feature_names,labels):
        
        self.X = X
        self.num_of_nodes = 0
        self.currentsplit = 0
        self.split_result = 0
        self.feature_names = feature_names #coloum names
        self.labels = labels#y
        self.catagories = set(labels)
        self.nodes = []
        self.split = 0
        self.leaf = 0
        self.steps = [0,0,0]
        

In [255]:
class Node:
    def __init__(self,labels,X,feature_ids):
        
        self.split_result = 0
        self.split = 0 #which column / which feature id
        self.feature_ids = feature_ids
        self.labels = labels
        self.X = X
        self.nodes = []
        self.regr = 0
        self.depth = 0
        self.top = 0
        self.bottom = 0
        self.steps = [0,0,0]
        
        self.leaf = 0 #true or false

In [256]:
def get_features(X,labels,column,bottom,top):#return splited data
    
    #get data in a certain range

    sub_label = [labels[x] for x in range(len(labels)) if X[x][column]<top and X[x][column]>=bottom]
    sub_X = [X[x] for x in range(len(X)) if X[x][column]<top and X[x][column]>=bottom]

        
    return [sub_X,sub_label]

In [257]:
def regression(regr_type,X,y,column,dgr):
    
    
    #reshape data
    
    X = [ [X[i][column]] for i in range(len(X)) ]

    
    regr = LinearRegression()
    regr.fit(X,y)
    
    
    return regr

In [258]:
def calculate_error(regr,X,y,column,regr_type,dgr):
    
    
    #reshape data
    X = [ [X[i][column]] for i in range(len(X)) ]
    if(regr_type == 1):
        X = PolynomialFeatures(degree=dgr).fit_transform(X)
    predict = regr.predict(X)

    difference = []
    
    for i in range(len(predict)):
        difference.append( (predict[i]-y[i])**2 )
    error = sum(difference)/len(difference)

  
    return error

In [259]:
def calculate_distance(i,j,column):
    

 
    n1 = [ i[0][k][column] for k in range(len(i[0])) ]
    n2 = [ j[0][k][column] for k in range(len(j[0])) ]
    
    
    
    Ni = len(n1)
    Nj = len(n2)
    
    s1i = sum(n1)/Ni
    s1j = sum(n2)/Nj
    
    s2i = sum([x**2 for x in n1])/Ni
    s2j = sum([x**2 for x in n2])/Nj
    
    s3i = sum([x*y for x,y in zip(n1,i[1])])/Ni
    s3j = sum([x*y for x,y in zip(n2,j[1])])/Nj
    
    s4i = sum(i[1])/Ni
    s4j = sum(j[1])/Nj
    
    D = (s1i-s1j)**2 + (s2i-s2j)**2 + (s3i-s3j)**2 +(s4i-s4j)**2
    
    return D

In [260]:
def find_min_distance(sliced_data,column):
    
    dist = 0
    min_dist = -1
    index1 = -1
    index2 = -1
    
    
    for i in range(len(sliced_data)-1):

            dist = calculate_distance(sliced_data[i],sliced_data[i+1],column)
            
            if(min_dist == -1):
                min_dist = dist
                index1 = i
                index2 = i+1
            if(dist<min_dist):
                min_dist = dist
                index1 = i
                index2 = i+1
    
    return index1,index2
    

In [261]:
def merge(sliced_data,regressions,column):
    
   

    index1,index2 = find_min_distance(sliced_data,column)
    

    sliced_data[index1][0] = sliced_data[index1][0]+sliced_data[index2][0]
    sliced_data[index1][1] = sliced_data[index1][1]+sliced_data[index2][1]

    regressions[index1] = regression(0,sliced_data[index1][0],sliced_data[index1][1],column,0)
    
    sliced_data.pop(index2)
    regressions.pop(index2)
    
    return sliced_data,regressions

In [262]:
def find_best_split(X,labels,feature_ids,steps):
    
    step = -1

    features = 0 
    svalue = 0
    
    min_error = -1
    min_sliced_data = []
    min_regressions = []
    min_steps = -1
    min_column = -1

    for column in feature_ids:

            
            X,labels = sort_f(X,labels,column)
            features = [ X[j][column] for j in range(len(X)) ] 
            
            sliced_data = []
            regressions = []
           
            
            dist = abs(features[0]-features[-1])
            
            
            step = int(dist/10)
            
            if(step < 2):
                step = 1

                a = [2,3]
                b = [0,1]
                if(steps[column] > 1 ):
                    continue

            for k in range(int(features[0]),int(features[-1])+1,step): 

                sliced_data.append(get_features(X,labels,column,k,k+step))
                
                if( len(sliced_data[-1][0]) == 0):

                    
                    sliced_data.pop(-1)
                    continue
                regressions.append( regression( 0,sliced_data[-1][0],sliced_data[-1][1],column,0 ) )
                
            while(len(sliced_data)>4):

                sliced_data,regressions = merge(sliced_data,regressions,column)
                
            
            error = 0
            for i in range(len(sliced_data)):
                
                error += calculate_error(regressions[i],sliced_data[i][0],sliced_data[i][1],column,0,0)
            #sum the error after the split
            #choose min
            
            if(min_error == -1):
                min_sliced_data = sliced_data
                min_regressions = regressions
                min_steps = step
                min_column = column
                min_error = error
                continue
                
            if(error<min_error):
                min_sliced_data = sliced_data
                min_regressions = regressions
                min_steps = step
                min_column = column
    
    #return split,split_value
    if(min_steps < 2 and min_steps>=0):
            steps[min_column] += 1
    
    return min_sliced_data,min_regressions,min_column,steps


In [263]:
def build_tree(X,feature_ids,labels,leaf_size,var,depth,steps):
       
    
    
    split = 0 
    sub_nodes = []
    
    
    st = steps.copy()
    
    
    split_data,split_regression,column,st = find_best_split(X,labels,feature_ids,st)
   
    for i,j in zip(split_data,split_regression):

        
        node = Node(i[1],i[0],feature_ids)
        node.split = column   #column
        
        #split point
       
        node.split_result = i[0][-1][column]
        node.depth = depth
        node.regr = j
        
        node.top = i[0][-1][column]
        node.bottom = i[0][0][column]
        node.steps = st
 
        
        
        #Ha egy bizonyos error érték alá megyünk vagy elértünk egy bizonyos elemszámot akkor a node-ot leaf-nek nyilvánítjuk
        if(len(node.labels) <= leaf_size or calculate_error(node.regr,node.X,node.labels,split,0,3)<1):
            node.leaf = 1
            
            
        else:
            node.leaf = 0
        sub_nodes.append(node)
    

    if(depth == 2000):
        for i in sub_nodes:
            i.leaf = 1
            
        return sub_nodes
    
    depth +=1

    
    leaf =  0
    for i in sub_nodes:
        #print(i.steps)
        if(i.leaf == 1):
            leaf +=1
    #if we reach limit return nodes             
            
    if(leaf == len(sub_nodes)):

        return sub_nodes
    
    else:
        for node in sub_nodes:

            if(node.leaf == 0):
                
                node.nodes = build_tree(node.X,node.feature_ids,node.labels,leaf_size,var,depth,node.steps)


    return sub_nodes   

In [264]:
def inicialize(X,feature_names,labels,leaf_size,var,depth):
    
    
    feature_ids = [x for x in range(len(feature_names))]
    tree = Tree(X,feature_names,labels)
    steps = []
    for i in feature_ids:
        steps.append(0)
    tree.nodes = build_tree(X,feature_ids,labels,leaf_size,var,depth,steps)
    return tree

In [ ]:
forest = []
times = []


for i in all_data:
    
    print("-------------------------------------------------------")

    start = time.time()
    tree = inicialize(i[0],i[-1],i[2],5,i,1)
    end = time.time()
    forest.append(tree)
    print(end-start)
    times.append(end-start)

In [278]:

def predict(node,value):

    if(node.leaf == 1 ):
       
        p = node.regr.predict(np.array(value[node.split]).reshape((1,-1)))
        return p
    else:
        
        c=0
        for i in node.nodes:

            if (i.top>value[i.split] and i.bottom<=value[i.split]):

                c+=1
                p=predict(i,value)
                return p
        if(c == 0):

            try:
                p = node.regr.predict(np.array(value[node.split]).reshape((1,-1)))
                return p
            except:
                return [-1]
            p = node.regr.predict(np.array(value[node.split]).reshape((1,-1)))
            return p



In [284]:
#Kiszámítjuk a teszt és a training data alapján az error mértékét
#Valamint a test adat alapján összehasonlítjuk az eredménnyel

diff = []#predicted points
difference = []
sum_y = []



for i,j in zip(forest,all_data):

    X_test = j[1]
    y_test = j[3]
    d = []
    error = 0
    for k in range(len(X_test)): 

        p = predict(i,X_test[k])

        if(p == None):
            d.append(0)
            continue
        #A few extreme situations appear, so I do not take them into consideration, in order to get a clear view during visualization
        if(abs(y_test[k]-p[0])<1000 and p[0]>0):
            error += abs(y_test[k]-p)
            d.append(p)
        else:
            d.append(0)
        
        #print(y_test[j],p)
        
    sum_y.append(sum(y_test))
    diff.append(d)
    difference.append(error)

print(difference)
print(sum_y)


with open("DataSets/difference.csv", 'w',newline='') as f:
    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    for row in difference:
        writer.writerow(row)
        
with open("DataSets/sum.csv", 'w',newline='') as f:
    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    for row in sum_y:
        writer.writerow([row])

[array([1106.63725695]), array([3.2661282]), array([630.04366401]), array([15191.60879266])]
[6095, 60, 5762.0, 622273.8784469217]


In [267]:
with open("DataSets/times.csv", 'w',newline='') as f:
    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    writer.writerow(times)